In [0]:
%sql
SHOW SCHEMAS;

databaseName
bronze
default
gold
information_schema
silver


In [0]:
spark.sql("""
SELECT *
FROM silver.customer_transactions
WHERE country = 'India'
""").explain(True)


== Parsed Logical Plan ==
'Project [*]
+- 'Filter ('country = India)
   +- 'UnresolvedRelation [silver, customer_transactions], [], false

== Analyzed Logical Plan ==
customer_id: bigint, age: bigint, gender: string, country: string, annual_income: bigint, spending_score: bigint, num_purchases: bigint, avg_purchase_value: double, membership_years: bigint, website_visits_per_month: bigint, cart_abandon_rate: double, churned: bigint, feedback_text: string, last_purchase_date: date, ingestion_ts: timestamp
Project [customer_id#13307L, age#13308L, gender#13309, country#13310, annual_income#13311L, spending_score#13312L, num_purchases#13313L, avg_purchase_value#13314, membership_years#13315L, website_visits_per_month#13316L, cart_abandon_rate#13317, churned#13318L, feedback_text#13319, last_purchase_date#13320, ingestion_ts#13321]
+- Filter (country#13310 = India)
   +- SubqueryAlias workspace.silver.customer_transactions
      +- Relation workspace.silver.customer_transactions[customer_id#

In [0]:
spark.sql("""
CREATE TABLE IF NOT EXISTS silver.customer_transactions_part
USING DELTA
PARTITIONED BY (country)
AS
SELECT * FROM silver.customer_transactions
""")


DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql("DESCRIBE DETAIL silver.customer_transactions_part").show(truncate=False)

+------+------------------------------------+-------------------------------------------+-----------+--------+-----------------------+-------------------+----------------+-----------------+--------+-----------+------------------------------------------------------------------------------+----------------+----------------+-----------------------------------------+---------------------------------------------------------------+-------------+
|format|id                                  |name                                       |description|location|createdAt              |lastModified       |partitionColumns|clusteringColumns|numFiles|sizeInBytes|properties                                                                    |minReaderVersion|minWriterVersion|tableFeatures                            |statistics                                                     |clusterByAuto|
+------+------------------------------------+-------------------------------------------+-----------+--------+--

In [0]:
spark.sql("""
OPTIMIZE silver.customer_transactions_part
ZORDER BY (customer_id)
""")

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,clusteringStats:struct<inputZCubeFiles:struct<numFiles:bigint,size:bigint>,inputOtherFiles:struct<numFiles:bigint,size:bigint>,inputNumZCubes:bigint,mergedFiles:struct<numFiles:bigint,size:bigint>,numOutputZCubes:bigint>,numBins:bigint,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,

In [0]:
import time

start = time.time()
spark.sql("""
SELECT *
FROM silver.customer_transactions_part
WHERE country = 'India'
""").count()

print(f"Query Time: {time.time() - start:.2f} seconds")

Query Time: 0.59 seconds


In [0]:
cached_df = spark.table("silver.customer_transactions_part")
cached_df.count()

10000